---
title: "KL-Regularized Least Squares"
subtitle: "Physics data"
authors:
  - Nicholas Barnield
  - Michael P. Friedlander
  - Tim Hoheisel
date: last-modified
# format: typst
format: html
execute:
    cache: true
---

In [ ]:
#| echo: false
#| output: false 
import Pkg; Pkg.activate("."); Pkg.instantiate()

In [ ]:
#| echo: false
#| code-fold: true
#| code-summary: "Import packages" 
using KLLS
using NPZ
using Plots
using Printf
using UnPack
using LinearAlgebra

## Load physics data

The file `PhysicsData.npz` contains the matrix and vectors generated by the script `PhysicsData.py`, where we get the output from `generate_data`:

- `A` is the first output argument `A`
- `b` is the second output argument `data0`
- `x0` is the third input argument, which seems to be the ground truth distribution.


In [ ]:
#| output: false 
data = npzread("./data/PhysicsData.npz", ["A", "b", "x0"])
@unpack A, b, x0 = data
klprob = KLLSData(A, b, name="Physics Data")

### Solve over a range of regularization parameters 

Solve the problem for a range of logarithmically spaced regularization parameters $\lambda$ between $10^{-8}$ and $1$.


In [ ]:
stats = map(exp10.(range(-16, stop=-5, length=3))) do λ
          klprob.λ = λ
          p, y, stats = newtoncg(klprob)
          (λ=λ, p=p, iters=stats.iter, ∇dNrm=stats.dual_feas)
        end;

Plot the recovered distributions for each value of $\lambda$ and overlay the ground truth distribution $x_0$.


In [ ]:
lab = hcat([@sprintf("λ = %6.0e", λ) for λ in getfield.(stats, :λ)]...)
default(lw=2, title="true and recovered distributions")
plot(getfield.(stats, :p), label=lab)
plot!(x0, label="ground truth")

The curve corresponding to the smallest parameter λ (1e-8) best approximates the modes of the ground-truth distribution, but smaller values of λ don't help.

These tests use a uniform prior. Does the data generator make a prior available? If so, this could be used to improve the results.

### Accurate solution 

Now try to solve the problem as accurately as possible. Set $\lambda=0$ and the tightest tolerance reasonable.


In [ ]:
#| echo: true
klprob.λ = 0.0
p, y, stats = newtoncg(klprob, atol=1e-12, rtol=1e-12, logging=0);

which achieves the following results:

In [ ]:
#| echo: false
m, n = size(A)
@printf("%20s: %11.2e\n", "rms(p-x0)", norm(p - x0)/√n)
@printf("%20s: %11.2e\n", "rms(Ap-b)", norm(A*p - b)/√m)
@printf("%20s: %11.2e\n", "rms(Ax0-b)", norm(A*x0 - b)/√m)
@printf("%20s: %11f\n", "Solve time", stats.elapsed_time)
@printf("%20s: %11d\n", "Number of iterations", stats.iter)
plot([p x0], lab=["x" "x0"], title="true vs recovered distributions")

## Synthetic data


In [ ]:
data = npzread("data/synthetic_data.npz", ["A", "b", "x"])
@unpack A, b, x = data
x0 = x / sum(x)
b = A*x0
klprob = KLLSData(A, b, name="Synthetic Data")

In [ ]:
klprob.λ = 0.0
p, y, stats = newtoncg(klprob, logging=3);

In [ ]:
norm(p - x0)/√length(x0)
plot([p x0])

In [ ]:
data = npzread("data/Tobias_data.npz", ["A", "b", "rho"])
@unpack A, b, rho = data
rho = convert(Vector{Float64}, rho)
rho = rho / sum(rho) 
b = A*rho
klprob = KLLSData(A, b, name="Tobias Data")

In [ ]:
p, y, stats = newtoncg(klprob, logging=1);
plot([p x0])